In [15]:
import pandas as pd
import json
from nltk.stem import PorterStemmer

import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /Users/taj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
import torch
from nltk.stem import PorterStemmer


def clean_text(text: str, stemmer: PorterStemmer, stop_words: list) -> list:
    lowered_text = text.lower()
    list_text = lowered_text.split(" ")
    clean_text = [stemmer.stem(word) for word in list_text if word not in stop_words]
    return clean_text


def bag_of_words(tokenized_text: list, bag: list) -> torch.Tensor:
    feature_vec = torch.zeros(len(bag))
    for idx, word in enumerate(bag):
        if word in tokenized_text:
            feature_vec[idx] = 1
        else:
            continue
    return feature_vec


In [3]:
with open("./message_data.json", "r") as f:
    data = json.load(f)

In [4]:
print(data)

{'messages': [{'tag': 'greeting', 'keywords': ['Hello', 'Hi', 'Hey'], 'responses': ["Hi there, nice to meet you! Hope you're having a nice day. :)"]}, {'tag': 'darktrace', 'keywords': ['What did you do at Darktrace?', 'What were your responsibilities at Darktrace?'], 'responses': ['At Darktrace I: <br> - Developed a lead scoring model to rank leads brought in through different channels. <br> - Developed a forecasting model to determine lead requirements from different channels. <br> - Data mining and created dashboards on PowerBi for stakeholders.']}, {'tag': 'hobbies', 'keywords': ['What do you like to do in your free time?', 'What are some of your hobbies?', 'Whare are your interests?'], 'responses': ['In my spare time I like to do things that keep me fit so I can eat more when I go out! :D I enjoy bouldering/ rock climbing/ snooker and cooking.']}, {'tag': 'technologies', 'keywords': ['What technologies do you know?', "What's your data science stack?"], 'responses': ["I feel comfort

In [5]:
data["messages"]

[{'tag': 'greeting',
  'keywords': ['Hello', 'Hi', 'Hey'],
  'responses': ["Hi there, nice to meet you! Hope you're having a nice day. :)"]},
 {'tag': 'darktrace',
  'keywords': ['What did you do at Darktrace?',
   'What were your responsibilities at Darktrace?'],
  'responses': ['At Darktrace I: <br> - Developed a lead scoring model to rank leads brought in through different channels. <br> - Developed a forecasting model to determine lead requirements from different channels. <br> - Data mining and created dashboards on PowerBi for stakeholders.']},
 {'tag': 'hobbies',
  'keywords': ['What do you like to do in your free time?',
   'What are some of your hobbies?',
   'Whare are your interests?'],
  'responses': ['In my spare time I like to do things that keep me fit so I can eat more when I go out! :D I enjoy bouldering/ rock climbing/ snooker and cooking.']},
 {'tag': 'technologies',
  'keywords': ['What technologies do you know?',
   "What's your data science stack?"],
  'responses'

In [10]:
df_list = []
for dictt in data["messages"]:
    print(dictt)
    df = pd.DataFrame({"keywords" : dictt["keywords"]})
    df["question_type"] = dictt["tag"]
    df_list.append(df)
cleaned_df = pd.concat(df_list, axis=0).reset_index(drop=True)

{'tag': 'greeting', 'keywords': ['Hello', 'Hi', 'Hey'], 'responses': ["Hi there, nice to meet you! Hope you're having a nice day. :)"]}
{'tag': 'darktrace', 'keywords': ['What did you do at Darktrace?', 'What were your responsibilities at Darktrace?'], 'responses': ['At Darktrace I: <br> - Developed a lead scoring model to rank leads brought in through different channels. <br> - Developed a forecasting model to determine lead requirements from different channels. <br> - Data mining and created dashboards on PowerBi for stakeholders.']}
{'tag': 'hobbies', 'keywords': ['What do you like to do in your free time?', 'What are some of your hobbies?', 'Whare are your interests?'], 'responses': ['In my spare time I like to do things that keep me fit so I can eat more when I go out! :D I enjoy bouldering/ rock climbing/ snooker and cooking.']}
{'tag': 'technologies', 'keywords': ['What technologies do you know?', "What's your data science stack?"], 'responses': ["I feel comfortable with a few o

In [11]:
cleaned_df.head()

,keywords,question_type
0,Hello,greeting
1,Hi,greeting
2,Hey,greeting
3,What did you do at Darktrace?,darktrace
4,What were your responsibilities at Darktrace?,darktrace


In [12]:
cleaned_df.iloc[2]

keywords              Hey
question_type    greeting
Name: 2, dtype: object

In [19]:
cleaned_df["no_punc"] = cleaned_df["keywords"].str.replace(
        r"[^[a-zA-Z\s]", "", regex=True
    )

In [20]:
cleaned_df.head()

,keywords,question_type,cleaned_text,no_punc
0,Hello,greeting,[hello],Hello
1,Hi,greeting,[hi],Hi
2,Hey,greeting,[hey],Hey
3,What did you do at Darktrace?,darktrace,[darktrace?],What did you do at Darktrace
4,What were your responsibilities at Darktrace?,darktrace,"[respons, darktrace?]",What were your responsibilities at Darktrace


In [21]:
cleaned_df["cleaned_text"] = cleaned_df["no_punc"].apply(clean_text, args=(PorterStemmer(), stopwords.words('english')))

In [23]:
cleaned_df.head(20)

,keywords,question_type,cleaned_text,no_punc
0,Hello,greeting,[hello],Hello
1,Hi,greeting,[hi],Hi
2,Hey,greeting,[hey],Hey
3,What did you do at Darktrace?,darktrace,[darktrac],What did you do at Darktrace
4,What were your responsibilities at Darktrace?,darktrace,"[respons, darktrac]",What were your responsibilities at Darktrace
5,What do you like to do in your free time?,hobbies,"[like, free, time]",What do you like to do in your free time
6,What are some of your hobbies?,hobbies,[hobbi],What are some of your hobbies
7,Whare are your interests?,hobbies,"[whare, interest]",Whare are your interests
8,What technologies do you know?,technologies,"[technolog, know]",What technologies do you know
9,What's your data science stack?,technologies,"[what, data, scienc, stack]",Whats your data science stack


In [26]:
set(cleaned_df["cleaned_text"].sum())

{'birthday',
 'colour',
 'darktrac',
 'data',
 'favourit',
 'free',
 'hello',
 'hey',
 'hi',
 'hobbi',
 'imperi',
 'interest',
 'know',
 'like',
 'respons',
 'scienc',
 'stack',
 'studi',
 'technolog',
 'time',
 'warwick',
 'whare',
 'what'}

In [ ]:
s